In [6]:
#Perry Gonzalez Issue 5
import pandas as pd
import sqlite3
# Question 1: Which teams have most consistently had the highest score?
con = sqlite3.connect("data/nba.sqlite")
overall = pd.read_sql_query("SELECT * FROM game LIMIT 100", con)
highest_score = pd.read_sql_query("SELECT team_name_home AS Team_Name, AVG(pts_home) AS Average_Points FROM game GROUP BY team_id_home ORDER BY AVG(pts_home) DESC", con)
highest_score

OperationalError: unable to open database file

In [3]:
# Question 2: Which team has had below average for the most 
# consecutive seasons?

total_average_pts = pd.read_sql_query("SELECT AVG(pts_home + pts_away) AS Average_Points FROM game", con)
#Total Avearage is 205.610704
below_average = pd.read_sql_query("""
    SELECT 
        team_name_home AS Team_Name, 
        team_id_home AS Team_ID,
        COUNT(season_id) AS Seasons_Below_Average
    FROM (
        SELECT 
            season_id, 
            team_id_home, 
            team_name_home,
            AVG(pts_home + pts_away) AS Season_Avg_Points
        FROM game
        GROUP BY season_id, team_id_home
        HAVING Season_Avg_Points < 205.610704
    ) AS Season_Below_Average
    GROUP BY team_id_home
    ORDER BY Seasons_Below_Average DESC
""", con)

below_average

NameError: name 'con' is not defined

In [4]:
# Question 3: Which team has consistently made it to the playoffs?

consistent_playoffs =  pd.read_sql_query("""
SELECT team_name_home AS team, COUNT(DISTINCT season_id) AS playoff_seasons
FROM game
WHERE season_type = 'Playoffs'
GROUP BY team
ORDER BY playoff_seasons DESC;
""", con)
consistent_playoffs

NameError: name 'con' is not defined

In [5]:
# Question 4: Does a higher proportion of successful three-point throws
# increase the likelihood of winning?

three_point_ratio_win_rate = pd.read_sql_query("""
WITH GameStats AS (
    SELECT 
        game_id,
        team_name_home AS team,
        fga_home, 
        fg3a_home,
        CAST(fg3a_home AS FLOAT) / NULLIF(fga_home, 0) AS three_pt_ratio,
        CASE WHEN wl_home = 'W' THEN 1 ELSE 0 END AS win
    FROM game
    WHERE fga_home IS NOT NULL AND fg3a_home IS NOT NULL
)
SELECT 
    CASE 
        WHEN three_pt_ratio < 0.2 THEN 'Low (<20%)'
        WHEN three_pt_ratio BETWEEN 0.2 AND 0.4 THEN 'Medium (20-40%)'
        WHEN three_pt_ratio BETWEEN 0.4 AND 0.6 THEN 'High (40-60%)'
        ELSE 'Very High (>60%)'
    END AS three_point_ratio_range,
    AVG(win) AS win_rate,
    AVG(three_pt_ratio) AS avg_three_pt_ratio
FROM GameStats
GROUP BY three_point_ratio_range
ORDER BY avg_three_pt_ratio DESC;
""", con)

three_point_ratio_win_rate

NameError: name 'con' is not defined

In [ ]:
# Question 5: How does the frequency of turnovers in a game affect
# the likelihood of winning? Simply put, do more turnovers help or hinder?

turnover_impact = pd.read_sql_query("""
WITH TurnoverStats AS (
    SELECT 
        game_id,
        team_name_home AS team,
        tov_home AS turnovers,
        CASE WHEN wl_home = 'W' THEN 1 ELSE 0 END AS win
    FROM game
    WHERE tov_home IS NOT NULL
)
SELECT 
    CASE 
        WHEN turnovers < 10 THEN 'Low (<10)'
        WHEN turnovers BETWEEN 10 AND 15 THEN 'Medium (10-15)'
        WHEN turnovers BETWEEN 16 AND 20 THEN 'High (16-20)'
        ELSE 'Very High (>20)'
    END AS turnover_range,
    AVG(win) AS win_rate,
    AVG(turnovers) AS avg_turnover_frequency
FROM TurnoverStats
GROUP BY turnover_range
ORDER BY avg_turnover_frequency DESC;
""", con)

turnover_impact